In [ ]:
%run -i 'cosim_import.py'

In [ ]:
caseH = 18
# --- EV Aggregator ---
sse = ev_ssm(ts=caseH, N=50000, step=1, tp=40,
             lr=0.1, lp=60, seed=2022, name="EVA")
sse.load_A("Aest.csv")

# --- ANDES case ---
dir_path = os.path.abspath('..')
case_path = '/case/ieee39_ev2.xlsx'
case = dir_path + case_path
ssa = andes.load(case,
                 setup=True,
                 no_output=True,
                 default_config=False)

# Set output mode as 'manual', turn off TDS progress bar
ssa.TDS.config.save_mode = 'manual'
ssa.TDS.config.no_tqdm = 1

# Set load as constant load.
ssa.PQ.config.p2p = 1
ssa.PQ.config.q2q = 1
ssa.PQ.config.p2z = 0
ssa.PQ.config.q2z = 0
ssa.PQ.pq2z = 0

# Turn on ``numba`` can accelerate TDS.
ssa.config.numba = 1

ssa.PFlow.run()

In [ ]:
ssa.TDS.config.tf = 2
ssa.TDS.run()

In [ ]:
ssa.DG.set(src='Pext0', attr='v', value=1, idx=ssa.EV2.idx.v)
ssa.TurbineGov.set(src='paux0', attr='v', value=1, idx='TGOV1_1')
ssa.TDS.config.tf = 4
ssa.TDS.run()

ssa.DG.set(src='Pext0', attr='v', value=2, idx=ssa.EV2.idx.v)
ssa.TurbineGov.set(src='paux0', attr='v', value=2, idx='TGOV1_1')
ssa.TDS.config.tf = 6
ssa.TDS.run()

ssa.DG.set(src='Pext0', attr='v', value=0, idx=ssa.EV2.idx.v)
ssa.TurbineGov.set(src='paux0', attr='v', value=0, idx='TGOV1_1')
ssa.TDS.config.tf = 20
ssa.TDS.run()

In [ ]:
ssa.TDS.plt.plot([ssa.EV2.Ipout_y, ssa.TGOV1N.pout], a=[0, 0])